In [31]:
import pandas as pd
import numpy as np

In [32]:
path="EquiposHUV-Zona4-copia.xlsx"
df=pd.read_excel(path)
df.head()

,ID,Nombre,Descripcion adicional,Marca,Modelo,Serie,Codigo actual,Codigo antiguo,Registro Sanitario,Estado actual,...,Descripción del ultimo correctivo,Cantidad de correctivos registrados,Cantidad de Tickets generados,Distribución por zonas,Información de contactos,Archivo Excel cargado de Hoja de vida,Tienen repuesto pendiente,Vida util,Guia rapida,Url del manual
0,1921,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,sn: ASFF-0075,NaN,NaN,NaN,Fuera de servicio,...,NaN,0,1,ZONA4(SEBASTIAN),NaN,NaN,si,5 AÑOS,NO,NaN
1,2061,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,sn: DXF82003081,NaN,NaN,NaN,Activo,...,NaN,0,0,ZONA4(SEBASTIAN),NaN,NaN,no,5 AÑOS,NO,NaN
2,2258,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,sn: ASFF-0073,NaN,NaN,NaN,Activo,...,NaN,0,0,ZONA4(SEBASTIAN),NaN,NaN,no,5 AÑOS,NO,NaN
3,2259,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,sn: DXF85003114,NaN,NaN,NaN,Fuera de servicio,...,NaN,0,1,ZONA4(SEBASTIAN),NaN,NaN,no,5 AÑOS,NO,NaN
4,2260,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,sn: DXF85003113,NaN,NaN,NaN,Fuera de servicio,...,"SEGUN ORDEN 4693 ""ANALIZADOR QUEDA EN MODO DE ...",1,1,ZONA4(SEBASTIAN),NaN,NaN,si,5 AÑOS,NO,NaN


In [33]:
df_salida=pd.DataFrame()
columnas_originales=df[["Nombre","Descripcion adicional","Marca","Modelo","Estado actual","Servicio","Clasificación biomedica","Clasificación de riesgo", "Cantidad de correctivos registrados"]]
df_salida=columnas_originales.copy()
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1


In [34]:
df_salida["Estado ponderado"]=np.where(df_salida["Estado actual"]=="Activo",1,np.where(df_salida["Estado actual"]=="Fuera de servicio",1,0))
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1


In [35]:
path2="datos/servicios.csv"
df_servicios=pd.read_csv(path2,sep=";",encoding='latin-1')
df_servicios.head()

,Servicio,Peso
0,UCI 2,3.0
1,MEDICA HOMBRES,2.0
2,MEDICA MUJERES,2.0
3,UCI,3.0
4,CIRUGIA PLASTICA,3.0


In [36]:
servicios_dic = df_servicios.set_index('Servicio').to_dict()["Peso"]
df_salida["Servicio ponderado"]=df_salida["Servicio"].apply(lambda x: servicios_dic[x])


In [37]:
clasificacion_dic={"TRATAMIENTO Y MANTENIMIENTO DE LA VIDA":3,
                    "RAHABILITACIÓN":2,"DIAGNOSTICO":2,
                    "APOYO":1,"N/R":1,"ANÁLISIS DE LABORATORIO":1,"REHABILITACIÓN":1,"NaN":1}
riesgo_dic={"I":1,"IIA":2,"IIB":3,"III":4,"NaN":1,"N/R":1}


In [38]:
df_salida["Clasificación biomedica"]=df_salida["Clasificación biomedica"].fillna("NaN")
df_salida["Clasificación de riesgo"]=df_salida["Clasificación de riesgo"].fillna("NaN")
df_salida["Clasificacion ponderada"]=df_salida["Clasificación biomedica"].apply(lambda x: clasificacion_dic[x])
df_salida["Riesgo ponderado"]=df_salida["Clasificación de riesgo"].apply(lambda x: riesgo_dic[x])
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado,Servicio ponderado,Clasificacion ponderada,Riesgo ponderado
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1,3.0,2,3
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1,3.0,2,2
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1,3.0,2,3
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1,3.0,2,3
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1,3.0,2,3


In [39]:
def func(x):
    if int(x)<=5:
        return 1
    elif int(x)<=9:
        return 2
    elif int(x)>10:
        return 3
    else:
        return 1

df_salida["Correctivos ponderados"]=df_salida["Cantidad de correctivos registrados"].apply(func)
df_salida.head()


,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado,Servicio ponderado,Clasificacion ponderada,Riesgo ponderado,Correctivos ponderados
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1,3.0,2,3,1
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1,3.0,2,2,1
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1,3.0,2,3,1
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1,3.0,2,3,1
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1,3.0,2,3,1


In [40]:
df_salida["Impacto operacional"]=1
df_salida["Solicitud por el personal"]=1

In [41]:
df_salida.head()

,Nombre,Descripcion adicional,Marca,Modelo,Estado actual,Servicio,Clasificación biomedica,Clasificación de riesgo,Cantidad de correctivos registrados,Estado ponderado,Servicio ponderado,Clasificacion ponderada,Riesgo ponderado,Correctivos ponderados,Impacto operacional,Solicitud por el personal
0,ANALIZADOR DE GASES,NaN,DRAGER,SCIO FOUR,Fuera de servicio,QUIROFANO 11,DIAGNOSTICO,IIB,0,1,3.0,2,3,1,1,1
1,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIA,0,1,3.0,2,2,1,1,1
2,ANALIZADOR DE GASES,NaN,DRAGER,Scio Four,Activo,QUIROFANO CENTRAL,DIAGNOSTICO,IIB,0,1,3.0,2,3,1,1,1
3,ANALIZADOR DE GASES,NaN,MINDRAY,MULTIGASES,Fuera de servicio,QUIROFANO 1- NEURO,DIAGNOSTICO,IIB,0,1,3.0,2,3,1,1,1
4,ANALIZADOR DE GASES,NaN,MINDRAY,NaN,Fuera de servicio,QUIROFANO 2- NEURO,DIAGNOSTICO,IIB,1,1,3.0,2,3,1,1,1


In [ ]:
def total(x):
    salida=x[9]*(x[10]*0.211+x[11]*0.033+x[12]*0.128+x[13]*0.06+x[14]*0.208+x[15]*0.359)
    return salida

df_salida["Total"]=df_salida.apply(total,axis=1)
df_salida.head()

In [47]:
import openpyxl


In [50]:
wbk=openpyxl.load_workbook("Datos_filtrados.xlsx")
for wks in wbk.worksheets:
    wks.cell(row=1,column=18).value="Total"
wbk.save("Datos_filtrados.xlsx")
wbk.close()

In [55]:
wbk=openpyxl.load_workbook("Datos_filtrados.xlsx")
wks=wbk.worksheets[0]
maxRow=wks.max_row
for row in range(2,maxRow):
    wks.cell(row=row,column=18).value="=K{}*(L{}*0.211+M{}*0.033+N{}*0.128+O{}*0.06+P{}*0.208+Q{}*0.359)".format(row,row,row,row,row,row,row)
wbk.save("Datos_filtrados.xlsx")
wbk.close()